In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
#Importing dataset

movies = pd.read_csv('ml-1m/movies.dat', sep= '::', header = None, engine = 'python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep= '::', header = None, engine = 'python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep= '::', header = None, engine = 'python', encoding='latin-1')

In [3]:
#training set and test set

training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype='int')

test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype='int')

In [4]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))

In [5]:
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [6]:
print(nb_users, nb_movies)

943 1682


In [7]:
def convert(data):
    print(data.shape)
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
training_set = convert(training_set)
test_set = convert(test_set)

(79999, 4)
(19999, 4)


In [9]:
pd.DataFrame(training_set)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
939,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [11]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()  
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [12]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr= 0.01, weight_decay = 0.5)

In [13]:
#Training

nb_epoch = 200

for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) >  0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print(f"epoch: {str(epoch)} loss: {str(train_loss/s)}")

epoch: 1 loss: tensor(1.7721)
epoch: 2 loss: tensor(1.0968)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0306)
epoch: 6 loss: tensor(1.0266)
epoch: 7 loss: tensor(1.0235)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0207)
epoch: 10 loss: tensor(1.0195)
epoch: 11 loss: tensor(1.0190)
epoch: 12 loss: tensor(1.0183)
epoch: 13 loss: tensor(1.0177)
epoch: 14 loss: tensor(1.0177)
epoch: 15 loss: tensor(1.0170)
epoch: 16 loss: tensor(1.0168)
epoch: 17 loss: tensor(1.0165)
epoch: 18 loss: tensor(1.0164)
epoch: 19 loss: tensor(1.0161)
epoch: 20 loss: tensor(1.0162)
epoch: 21 loss: tensor(1.0158)
epoch: 22 loss: tensor(1.0162)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0160)
epoch: 26 loss: tensor(1.0155)
epoch: 27 loss: tensor(1.0152)
epoch: 28 loss: tensor(1.0150)
epoch: 29 loss: tensor(1.0132)
epoch: 30 loss: tensor(1.0119)
epoch: 31 loss: tensor(1.0095)
epoch: 32 loss: tensor(1.0099)
epoch: 33 loss: t

In [14]:
#testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        s += 1.
print(f"test loss: {str(test_loss/s)}")

test loss: tensor(0.9548)
